# Reddit Trading Bot

# To get all posts from a subreddit

In [ ]:
import requests
from datetime import datetime
import traceback
import time
import json
import sys
import re
import pandas as pd

In [ ]:
df = pd.DataFrame(columns = ['Score', 'date', 'text', 'subreddit'])
df

In [ ]:
username = ""  # put the username you want to download in the quotes
subreddits_general =['tsla',
            'teslainvestorsclub',
            'TSLALounge',
            'TSLAtalk',
            'TSLAsexy',
            'Tesla_Stock',
            'tslaq']
subreddits_exec = [
                   'stocks',
                   'investing',
                   'pennystocks',
                   'robinhood',
                   'investing_discussion',
                   'cryptocurrency',
                   'Dividends',
                   'Dividends ',
                    'wallstreetbets'

                   ]
# From https://www.investopedia.com/reddit-top-investing-and-trading-communities-5189322
# put the subreddit you want to download in the quotes
# leave either one blank to download an entire user's or subreddit's history
# or fill in both to download a specific users history from a specific subreddit

url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&{}&before="

start_time = datetime.utcnow()

In [ ]:
def downloadFromUrl(object_type, fs):
    global df
    #print(f"Saving {object_type}s to {filename}")
    filter_string = fs
    print(fs," *********----------------started--------------***************")
    count = 0
    #handle = open(filename, 'w')
    previous_epoch = int(start_time.timestamp())
    while True:
        new_url = url.format(object_type, filter_string)+str(previous_epoch)
        json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
        time.sleep(1)  # pushshift has a rate limit, if we send requests too fast it will start returning error messages
        try:
            json_data = json_text.json()
        except json.decoder.JSONDecodeError:
            time.sleep(1)
            continue

        if 'data' not in json_data:
            break
        objects = json_data['data']
        if len(objects) == 0:
            break

        for object in objects:
            previous_epoch = object['created_utc'] - 1
            
            if object_type == 'comment':
                try:
                    '''handle.write(str(object['score']))
                    handle.write(" : ")
                    handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
                    handle.write("\n")
                    handle.write(object['body'].encode(encoding='ascii', errors='ignore').decode())
                    handle.write("\n-------------------------------\n")'''
                    tweet  = object['body'].encode(encoding='ascii', errors='ignore').decode()
                    if tweet.find("TSLA") >= 0 or tweet.find("TESLA") >= 0 or tweet.find("Tesla") >= 0 or tweet.find("tesla") >= 0 or tweet.find("tsla") >= 0 :
                        if object['score'] >= 0:
                            df = df.append({'Score' : object['score'],
                                            'date' : datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"),
                                            'text' : tweet,
                                            'subreddit':filter_string},
                                           ignore_index = True)
                            count += 1
                except Exception as err:
                    print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
                    print(traceback.format_exc())
            elif object_type == 'submission':
                if object['is_self']:
                    if 'selftext' not in object:
                        continue
                    try:
                        '''handle.write(str(object['score']))
                        handle.write(" : ")
                        handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
                        handle.write("\n")
                        handle.write(object['selftext'].encode(encoding='ascii', errors='ignore').decode())
                        handle.write("\n-------------------------------\n")'''
                        tweet = object['selftext'].encode(encoding='ascii', errors='ignore').decode()
                        
                        if tweet.find("TSLA") >= 0 or tweet.find("TESLA") >= 0 or tweet.find("Tesla") >= 0 or tweet.find("tesla") >= 0 or tweet.find("tsla") >= 0 :
                            if object['score'] >= 1:
                                df = df.append({'Score' : object['score'],
                                                'date' : datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"),
                                                'text' : tweet,
                                                'subreddit':filter_string},
                                               ignore_index = True)
                                count += 1
                    except Exception as err:
                        print(f"Couldn't print post: {object['url']}")
                        print(traceback.format_exc())

        print("Saved {} {}s through {} for {}".format(count, object_type, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d"), fs))

    print(f"Saved {count} {object_type}s")

    #handle.close()

In [ ]:
for subreddit in subreddits_general:
    filter_string = None
    if username == "" and subreddit == "":
        print("Fill in either username or subreddit")
        sys.exit(0)
    elif username == "" and subreddit != "":
        filter_string = f"subreddit={subreddit}"
    elif username != "" and subreddit == "":
        filter_string = f"author={username}"
    else:
        filter_string = f"author={username}&subreddit={subreddit}"
        
    downloadFromUrl("submission", filter_string)
    print(" "+subreddit+" ********------------------COMPLETED----------********************************")
   
    time.sleep(1)

In [ ]:
df.columns =['upvotes','Date','Post', 'sub']
subs =[]
for line in df['sub']:
  line = re.sub('subreddit=', '', line)
  subs.append(line)
df['sub'] = subs
df

In [ ]:
df['sub'].value_counts()

In [ ]:
df.drop_duplicates(subset =["Post"],
                     keep = 'first', inplace = True)
df

In [ ]:
df.to_csv('GENERALRedditTesla.csv', sep=',', index = False)
print('csv done')

In [ ]:
df = pd.DataFrame(columns = ['Score', 'date', 'text', 'subreddit'])
df

In [ ]:
for subreddit in subreddits_exec:
    filter_string = None
    if username == "" and subreddit == "":
        print("Fill in either username or subreddit")
        sys.exit(0)
    elif username == "" and subreddit != "":
        filter_string = f"subreddit={subreddit}"
    elif username != "" and subreddit == "":
        filter_string = f"author={username}"
    else:
        filter_string = f"author={username}&subreddit={subreddit}"
        
    downloadFromUrl("submission", filter_string)
    print(" "+subreddit+" ********------------------COMPLETED----------********************************")
    time.sleep(1.5)

In [ ]:
df.columns =['upvotes','Date','Post', 'sub']
subs =[]
for line in df['sub']:
  line = re.sub('subreddit=', '', line)
  subs.append(line)
df['sub'] = subs
df

In [ ]:
df.drop_duplicates(subset =["Post"],
                     keep = 'first', inplace = True)
df

In [ ]:
df['sub'].value_counts()

In [ ]:
df.to_csv('EXEC_RedditTesla.csv', sep=',', index = False)
print('csv done')